In [48]:
import psycopg2
from psycopg2.extras import RealDictCursor
import pandas as pd
import uuid
import requests
import json

In [43]:
conn = psycopg2.connect(
    host='ud-adp-rolling.cbyis8su6n0i.us-east-2.rds.amazonaws.com',
    user='postgres',
    password='liammurphybbm2',
    port=5432
)

cursor = conn.cursor(cursor_factory=RealDictCursor)

# Add BBM4 players to postgres db

# Add BBM5 players to Postgres db

In [8]:
data = pd.read_csv('data/bbm5_adp.csv', delimiter=',')

for idx, row in data.iterrows():
    cursor.execute("INSERT INTO player_info (player_id, player_name, position) VALUES (%s, %s, %s)",
                   (row['player_id'], row['player_name'], row['position_name']))
    
conn.commit()

In [15]:
for idx, row in data.iterrows():
    for date in data.columns[4:]:
        cursor.execute("INSERT INTO player_adp (player_id, date, adp) VALUES (%s, %s, %s)",
                    (row['player_id'], date, row[date]))

conn.commit()

# Add BBM6 players to postgres db

In [15]:
bbm6_data = pd.read_csv('data/Underdog_Draft_Table_2025-07-13.csv', delimiter=',')
bbm6_data.head()

,Rank,Player,Position,Position Rank,ADP on April 29,ADP on July 12,ADP Change
0,1,Ja'Marr Chase,WR,WR1,1.1,1.1,0.0
1,2,Justin Jefferson,WR,WR2,3.7,2.8,-0.9
2,3,Bijan Robinson,RB,RB1,3.7,3.1,-0.6
3,4,Saquon Barkley,RB,RB2,2.5,4.1,1.6
4,5,CeeDee Lamb,WR,WR3,5.3,4.8,-0.5


In [46]:
cursor.execute('SELECT player_name FROM player_info')
player_info = cursor.fetchall()

player_names = []
for player in player_info:
    player_names.append(player['player_name'])

player_data = []
for idx, row in bbm6_data.iterrows():
    if row['Player'] not in player_names:
        print(row['Player'])
        player_data.append((str(uuid.uuid4()), row['Player'], row['Position']))

cursor.executemany(
    "INSERT INTO player_info (player_id, player_name, position) VALUES (%s, %s, %s)",
    player_data
)

conn.commit()

In [47]:
conn.close()
cursor.close()